In [13]:
import numpy as np
import random
from citipy import citipy

In [14]:
# Create a practice set of random latitude and longitude combinations.
x = [25.12903645, 25.92017388, 26.62509167, -59.98969384, 37.30571269]
y = [-67.59741259, 11.09532135, 74.84233102, -76.89176677, -61.13376282]
coordinates = zip(x, y)

In [15]:
# Use the tuple() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

25.12903645 -67.59741259
25.92017388 11.09532135
26.62509167 74.84233102
-59.98969384 -76.89176677
37.30571269 -61.13376282


In [16]:
lats = np.random.uniform(-90 , 90 , size = 1500)

long = np.random.uniform(-180 , 180 , size = 1500)

lat_langs = zip(lats , long)

lat_langs

In [17]:
# Add the latitudes and longitudes to a list.
coordinates = list(zip(lat_langs))